In [137]:
import numpy as np
import pandas as pd
import model.preprocess_games as ppg
from pandas import DataFrame
pd.options.display.max_columns = None
pd.set_option("display.max_colwidth", -1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option("display.max_rows", 101)

/home/nathaniel/anaconda3/envs/nba-predictor/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [94]:
DATA_PATH = 'data'

SEASONS_PROCESSED_DS = f"{DATA_PATH}/seasons.processed.feather"

TEAMS_DS = f"{DATA_PATH}/teams.processed.feather"
TEAMS_PROCESSED_DS = f"{DATA_PATH}/teams.processed.feather"

GAMES_DS = f"{DATA_PATH}/games.csv"
GAMES_PROCESSED_DS = f"{DATA_PATH}/games.processed.feather"

In [95]:
games = pd.read_csv(GAMES_DS, parse_dates=["GAME_DATE_EST"], usecols=["GAME_ID",'GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
       'SEASON', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home',
       'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away',
       'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away',
       'HOME_TEAM_WINS']
                    , infer_datetime_format=True, index_col="GAME_ID")
games = games.sort_values(by=['GAME_DATE_EST', 'GAME_ID'])
teams = pd.read_feather(TEAMS_PROCESSED_DS)
seasons = pd.read_feather(SEASONS_PROCESSED_DS)
games_matchup = pd.read_feather(GAMES_PROCESSED_DS)
games_matchup.set_index(["GAME_ID"])

,GAME_DATE_EST,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,VT_OVERALL_DEF_3P,VT_OVERALL_OFF_FT,VT_OVERALL_DEF_FT,VT_OVERALL_OFF_REB,VT_OVERALL_DEF_REB,VT_AWAY_POINTS,VT_AWAY_FG,VT_AWAY_3P,VT_AWAY_FT,VT_AWAY_REB
GAME_ID,,,,,,,,,,,,,,,,,,,,,
21801229,2019-04-10,Final,1610612746,1610612762,2018,143.0,0.509,0.767,0.429,34.0,...,0.363,0.752,0.762,42.947,40.990,98.817,0.451,0.346,0.752,42.197
21801228,2019-04-10,Final,1610612743,1610612750,2018,99.0,0.448,0.647,0.303,23.0,...,0.365,0.781,0.763,42.933,42.824,102.950,0.451,0.337,0.785,42.397
21801227,2019-04-10,Final,1610612759,1610612742,2018,105.0,0.466,0.833,0.333,22.0,...,0.353,0.776,0.758,41.926,44.452,100.544,0.451,0.356,0.772,41.053
21801226,2019-04-10,Final,1610612749,1610612760,2018,116.0,0.430,0.714,0.326,27.0,...,0.352,0.774,0.758,45.669,41.648,105.281,0.454,0.340,0.773,45.628
21801225,2019-04-10,Final,1610612763,1610612744,2018,132.0,0.490,0.833,0.457,25.0,...,0.343,0.779,0.761,44.004,43.886,107.528,0.469,0.384,0.774,43.608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21600005,2016-10-26,Final,1610612754,1610612742,2016,130.0,0.505,0.765,0.526,30.0,...,0.349,0.780,0.751,42.036,44.170,99.992,0.455,0.359,0.779,41.420
21600004,2016-10-26,Final,1610612753,1610612748,2016,96.0,0.382,0.786,0.333,20.0,...,0.351,0.758,0.750,40.374,40.288,97.752,0.471,0.352,0.748,40.168
21600003,2016-10-25,Final,1610612744,1610612759,2016,100.0,0.471,0.722,0.212,24.0,...,0.349,0.778,0.763,42.819,41.653,101.618,0.467,0.365,0.779,42.256


In [96]:
games.info()
games.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23195 entries, 10300001 to 21900901
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   GAME_DATE_EST     23195 non-null  datetime64[ns]
 1   GAME_STATUS_TEXT  23195 non-null  object        
 2   HOME_TEAM_ID      23195 non-null  int64         
 3   VISITOR_TEAM_ID   23195 non-null  int64         
 4   SEASON            23195 non-null  int64         
 5   PTS_home          23096 non-null  float64       
 6   FG_PCT_home       23096 non-null  float64       
 7   FT_PCT_home       23096 non-null  float64       
 8   FG3_PCT_home      23096 non-null  float64       
 9   AST_home          23096 non-null  float64       
 10  REB_home          23096 non-null  float64       
 11  PTS_away          23096 non-null  float64       
 12  FG_PCT_away       23096 non-null  float64       
 13  FT_PCT_away       23096 non-null  float64       
 14  FG3_PCT_away

Index(['GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
       'SEASON', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home',
       'AST_home', 'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away',
       'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

In [138]:
df = games_matchup[games_matchup.SEASON==2018]
row = df.tail(1).iloc[0, :]
display(row)
type(row)

GAME_ID                  21800001           
GAME_DATE_EST            2018-10-16 00:00:00
GAME_STATUS_TEXT         Final              
HOME_TEAM_ID             1610612738         
VISITOR_TEAM_ID          1610612755         
SEASON                   2018               
PTS_home                 105                
FG_PCT_home              0.433              
FT_PCT_home              0.714              
FG3_PCT_home             0.297              
AST_home                 21                 
REB_home                 55                 
PTS_away                 87                 
FG_PCT_away              0.391              
FT_PCT_away              0.609              
FG3_PCT_away             0.192              
AST_away                 18                 
REB_away                 47                 
HOME_TEAM_WINS           1                  
HT                       1610612738         
HT_RANK                  None               
HT_CLASS                 None               
HT_HW     

pandas.core.series.Series

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1229 entries, 0 to 1228
Data columns (total 71 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   GAME_ID                1229 non-null   int64         
 1   GAME_DATE_EST          1229 non-null   datetime64[ns]
 2   GAME_STATUS_TEXT       1229 non-null   object        
 3   HOME_TEAM_ID           1229 non-null   int64         
 4   VISITOR_TEAM_ID        1229 non-null   int64         
 5   SEASON                 1229 non-null   int64         
 6   PTS_home               1229 non-null   float64       
 7   FG_PCT_home            1229 non-null   float64       
 8   FT_PCT_home            1229 non-null   float64       
 9   FG3_PCT_home           1229 non-null   float64       
 10  AST_home               1229 non-null   float64       
 11  REB_home               1229 non-null   float64       
 12  PTS_away               1229 non-null   float64       
 13  FG_

In [99]:
teams[teams.TEAM_ID == 1610612738]

,TEAM_ID,NAME,NICKNAME,CITY
1,1610612738,BOS,Celtics,Boston
